Imports

In [1]:
import pandas as pd
import re

carregando xlsx

In [2]:
df = pd.read_excel('laudos_piloto002.xlsx')

Padronizando Regiao

In [3]:
df['Regiao'] = df['Regiao'].str.upper()

In [4]:
df.loc[df['Regiao'].str.contains('ABDOM', na=False), 'Regiao'] = 'ABDOMEN'

In [5]:
def extrair_regioes(texto):
    regioes = []
    if pd.isna(texto):
        return []

    texto = texto.upper()

    if 'ABDOM' in texto or 'ABDÔMEN' in texto:
        regioes.append('ABDOMEN')
    if 'TÓRAX' in texto or 'CERVICAL (TECIDOS MOLES)' in texto or 'CERVICAL (PARTES MOLES)' in texto or 'TORAX' in texto:
        regioes.append('TORAX')
    if 'CRÂNIO' in texto:
        regioes.append('CRANIO')
    if any(x in texto for x in ['COLUNA', 'LOMBOSSACRA','TORACOLOMBAR','COCCÍGEO', 'CALDA']):
        regioes.append('COLUNA')
    if any(x in texto for x in ['COXAL', 'MEMBRO', 'JOELHO', 'FÊMUR','OMBRO', 'COTOVELO', 'RÁDIO', 'ULNA', 'TARSO', 'DIGITO', 'PELVE', 'MPE', 'MPD', 'MTD', 'MP', 'CONTRA']):
        regioes.append('EXTREMIDADES')
    if ('CORPO INTEIRO' in texto or 'SILVESTRE' in texto or re.search(r'\bCI\b', texto) or 'CAVIDADE CELOMÁTICA' in texto):
        regioes.append('CORPO_INTEIRO')
    if 'SONDA' in texto or 'ESOFAGICA' in texto:
        regioes.append('TORAX')

    return list(set(regioes)) 

isolando corpo inteiro

In [6]:
def ajustar_regiao_corpo(regioes):
    if 'CORPO_INTEIRO' in regioes:
        return ['CORPO_INTEIRO']
    return regioes

In [7]:
df['regiao_corpo'] = df['Regiao'].apply(extrair_regioes)
df['regiao_corpo'] = df['regiao_corpo'].apply(ajustar_regiao_corpo)
df['regiao_corpo_str'] = df['regiao_corpo'].apply(lambda x: ', '.join(x))


In [8]:
df['regiao_corpo_str'] = df['regiao_corpo'].apply(lambda x: ', '.join(x))

In [9]:
def preencher_regiao(row):
    # Se já tem regiao_corpo preenchido, mantém
    if row['regiao_corpo']:
        return row['regiao_corpo']
    
    regioes = []

    # Tentar extrair da Exames Separados
    if isinstance(row['Exames Separados'], list):
        for exame in row['Exames Separados']:
            regioes += extrair_regioes(exame)
    
    # Se ainda vazio, tenta de Todos Exames
    if not regioes and isinstance(row['Todos Exames'], str):
        regioes += extrair_regioes(row['Todos Exames'])

    # Eliminar duplicatas
    return list(set(regioes))


In [10]:
df['regiao_corpo'] = df.apply(preencher_regiao, axis=1)

In [11]:
df['regiao_corpo_str'] = df['regiao_corpo'].apply(lambda x: ', '.join(x))

In [12]:
df[df['regiao_corpo'].apply(lambda x: len(x) == 0)].shape

(0, 23)

In [13]:
df['regiao_corpo'] = df['regiao_corpo'].apply(lambda x: x if isinstance(x, list) and x else ['INDEFINIDO'])

Filtrando as categorias para melhorar a eficacia do modelo

In [14]:
categorias_validas = [
    ['TORAX'],
    ['CORPO_INTEIRO'],
    ['EXTREMIDADES'],
    ['COLUNA'],
    ['ABDOMEN'],
    ['CRANIO'],
    ['EXTREMIDADES', 'COLUNA']
]

In [15]:
def manter_categoria(regioes):
    return regioes in categorias_validas

In [16]:
df = df[df['regiao_corpo'].apply(manter_categoria)].reset_index(drop=True)

In [17]:
df['regiao_corpo'].value_counts()

regiao_corpo
[TORAX]            338
[EXTREMIDADES]     257
[COLUNA]           125
[CORPO_INTEIRO]    117
[ABDOMEN]           70
[CRANIO]            55
Name: count, dtype: int64

In [19]:
df.to_excel('laudos_piloto002.xlsx', index=False)